## Imports

In [1]:
import pandas as pd
import ast

## Datasets

In [2]:
credits = pd.read_csv('credits.csv')
movies = pd.read_csv('movies_metadata.csv')

/var/folders/dt/ywvbqwcj3hn_5x4g7z8p1tq00000gn/T/ipykernel_9859/1991119359.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv('movies_metadata.csv')


In [3]:
movies.head(1)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


## Cleaning

In [4]:
# remove unnecessary columns
movies.drop(columns=['belongs_to_collection', 'budget', 'homepage', 'imdb_id', 'overview', 'poster_path', 'runtime', 'status', 'tagline', 'video'], inplace=True)

In [5]:
movies.head(1)

,adult,genres,id,original_language,original_title,popularity,production_companies,production_countries,release_date,revenue,spoken_languages,title,vote_average,vote_count
0,False,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,Toy Story,21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Toy Story,7.7,5415.0


In [6]:
# function to check int id types
def is_integer(val):
    try:
        # try to convert to int
        int(val)
        return True
    except (ValueError, TypeError):
        return False

# filter rows where 'id' is an integer-like value
movies = movies[movies.id.apply(is_integer)]

# convert 'id' column to int
movies.id = movies.id.astype(int)

# merge with credits df
df = pd.merge(movies, credits, on='id')
df.drop(columns=['id'], inplace=True)

In [7]:
df.head(1)

,adult,genres,original_language,original_title,popularity,production_companies,production_countries,release_date,revenue,spoken_languages,title,vote_average,vote_count,cast,crew
0,False,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",en,Toy Story,21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Toy Story,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."


In [8]:
# extract the category names
def extract_names(name_str):
    if pd.isna(name_str):
        return []
    # convert the string representation of the list to an actual list
    str_list = ast.literal_eval(name_str)
    # extract the 'name' from each dictionary in the list
    names = [i['name'] for i in str_list]
    # return list of names as a string
    return ', '.join(names)

# clean the cast data
def clean_cast_data(cast_str):
    # convert string representation of the list to an actual list
    cast_list = ast.literal_eval(cast_str)

    # extract relevant fields and change gender values
    cleaned_cast = []
    for member in cast_list:
        cleaned_member = {
            'character': member['character'],
            'gender': 'm' if member['gender'] == 2 else 'f' if member['gender'] == 1 else None,
            'name': member['name'],
            'order': member['order']
        }
        cleaned_cast.append(cleaned_member)
    return cleaned_cast

# clean the crew data
def clean_crew_data(crew_str):
    # convert string representation of the list to an actual list
    crew_list = ast.literal_eval(crew_str)

    # extract relevant fields
    cleaned_crew = []
    for member in crew_list:
        cleaned_member = {
            'job': member['job'],
            'name': member['name']
        }
        cleaned_crew.append(cleaned_member)
    return cleaned_crew

# clean columns
cols = ['genres', 'production_companies', 'production_countries', 'spoken_languages']
for col in cols:
    df[col] = df[col].apply(extract_names)
df.release_date = pd.to_datetime(df.release_date)
df.cast = df.cast.apply(clean_cast_data)
df.crew = df.crew.apply(clean_crew_data)

In [9]:
df.head(1)

,adult,genres,original_language,original_title,popularity,production_companies,production_countries,release_date,revenue,spoken_languages,title,vote_average,vote_count,cast,crew
0,False,"Animation, Comedy, Family",en,Toy Story,21.946943,Pixar Animation Studios,United States of America,1995-10-30,373554033.0,English,Toy Story,7.7,5415.0,"[{'character': 'Woody (voice)', 'gender': 'm',...","[{'job': 'Director', 'name': 'John Lasseter'},..."


## Example Usages

In [10]:
df[df.original_title == 'Toy Story']

,adult,genres,original_language,original_title,popularity,production_companies,production_countries,release_date,revenue,spoken_languages,title,vote_average,vote_count,cast,crew
0,False,"Animation, Comedy, Family",en,Toy Story,21.946943,Pixar Animation Studios,United States of America,1995-10-30,373554033.0,English,Toy Story,7.7,5415.0,"[{'character': 'Woody (voice)', 'gender': 'm',...","[{'job': 'Director', 'name': 'John Lasseter'},..."


In [11]:
'Comedy' in df[df.original_title == 'Toy Story'].genres[0]

True

In [12]:
df[df['release_date'].dt.year == 2013]

,adult,genres,original_language,original_title,popularity,production_companies,production_countries,release_date,revenue,spoken_languages,title,vote_average,vote_count,cast,crew
1090,False,"Comedy, Horror",en,The Sleepover,0.135596,,,2013-10-12,0.0,,The Sleepover,8.0,1.0,"[{'character': 'Rachel', 'gender': None, 'name...","[{'job': 'Director', 'name': 'Chris Cullari'}]"
4370,False,Drama,fr,Camille Claudel 1915,0.134014,"Canal+, Arte France Cinéma, 3B Productions, C....",France,2013-03-13,115860.0,Français,Camille Claudel 1915,7.0,20.0,"[{'character': 'Camille Claudel', 'gender': 'f...","[{'job': 'Director', 'name': 'Bruno Dumont'}, ..."
4371,False,Drama,fr,Camille Claudel 1915,0.134014,"Canal+, Arte France Cinéma, 3B Productions, C....",France,2013-03-13,115860.0,Français,Camille Claudel 1915,7.0,20.0,"[{'character': 'Camille Claudel', 'gender': 'f...","[{'job': 'Producer', 'name': 'Rachid Bouchareb..."
4372,False,Drama,fr,Camille Claudel 1915,0.110065,"Canal+, Arte France Cinéma, 3B Productions, C....",France,2013-03-13,115860.0,Français,Camille Claudel 1915,7.0,20.0,"[{'character': 'Camille Claudel', 'gender': 'f...","[{'job': 'Director', 'name': 'Bruno Dumont'}, ..."
4373,False,Drama,fr,Camille Claudel 1915,0.110065,"Canal+, Arte France Cinéma, 3B Productions, C....",France,2013-03-13,115860.0,Français,Camille Claudel 1915,7.0,20.0,"[{'character': 'Camille Claudel', 'gender': 'f...","[{'job': 'Producer', 'name': 'Rachid Bouchareb..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45397,False,Comedy,fr,La Fille du 14 juillet,1.096974,Ecce Films,France,2013-06-05,0.0,Français,The Rendez-Vous of Déjà-Vu,6.2,13.0,"[{'character': 'Truquette', 'gender': 'f', 'na...","[{'job': 'Director', 'name': 'Antonin Peretjat..."
45414,False,Drama,en,Over/Under,0.704642,"Fox Television Studios, Universal Cable Produc...",United States of America,2013-01-04,0.0,English,Over/Under,4.0,2.0,"[{'character': '', 'gender': 'm', 'name': 'Ste...","[{'job': 'Director', 'name': 'Bronwen Hughes'}..."
45431,False,,en,What a Fuck Am I Doing on This Battlefield,0.020752,,,2013-04-22,0.0,,What a Fuck Am I Doing on This Battlefield,0.0,0.0,[],"[{'job': 'Director', 'name': 'Julien Fezans'},..."
45448,False,"Comedy, Romance",ru,Полярный рейс,0.036523,Wise Vision,Ukraine,2013-12-12,0.0,Pусский,Polar Flight,0.0,0.0,"[{'character': '', 'gender': None, 'name': 'Eg...","[{'job': 'Writer', 'name': 'Aleksandr Kachan'}..."


In [23]:
df[df.original_title == 'Toy Story'].cast.values[0]

[{'character': 'Woody (voice)',
  'gender': 'm',
  'name': 'Tom Hanks',
  'order': 0},
 {'character': 'Buzz Lightyear (voice)',
  'gender': 'm',
  'name': 'Tim Allen',
  'order': 1},
 {'character': 'Mr. Potato Head (voice)',
  'gender': 'm',
  'name': 'Don Rickles',
  'order': 2},
 {'character': 'Slinky Dog (voice)',
  'gender': 'm',
  'name': 'Jim Varney',
  'order': 3},
 {'character': 'Rex (voice)',
  'gender': 'm',
  'name': 'Wallace Shawn',
  'order': 4},
 {'character': 'Hamm (voice)',
  'gender': 'm',
  'name': 'John Ratzenberger',
  'order': 5},
 {'character': 'Bo Peep (voice)',
  'gender': 'f',
  'name': 'Annie Potts',
  'order': 6},
 {'character': 'Andy (voice)',
  'gender': None,
  'name': 'John Morris',
  'order': 7},
 {'character': 'Sid (voice)',
  'gender': 'm',
  'name': 'Erik von Detten',
  'order': 8},
 {'character': 'Mrs. Davis (voice)',
  'gender': 'f',
  'name': 'Laurie Metcalf',
  'order': 9},
 {'character': 'Sergeant (voice)',
  'gender': 'm',
  'name': 'R. Lee Erme

In [24]:
df[df.original_title == 'Toy Story'].crew.values[0]

[{'job': 'Director', 'name': 'John Lasseter'},
 {'job': 'Screenplay', 'name': 'Joss Whedon'},
 {'job': 'Screenplay', 'name': 'Andrew Stanton'},
 {'job': 'Screenplay', 'name': 'Joel Cohen'},
 {'job': 'Screenplay', 'name': 'Alec Sokolow'},
 {'job': 'Producer', 'name': 'Bonnie Arnold'},
 {'job': 'Executive Producer', 'name': 'Ed Catmull'},
 {'job': 'Producer', 'name': 'Ralph Guggenheim'},
 {'job': 'Executive Producer', 'name': 'Steve Jobs'},
 {'job': 'Editor', 'name': 'Lee Unkrich'},
 {'job': 'Art Direction', 'name': 'Ralph Eggleston'},
 {'job': 'Editor', 'name': 'Robert Gordon'},
 {'job': 'Foley Editor', 'name': 'Mary Helen Leasman'},
 {'job': 'Animation', 'name': 'Kim Blanchette'},
 {'job': 'ADR Editor', 'name': 'Marilyn McCoppen'},
 {'job': 'Orchestrator', 'name': 'Randy Newman'},
 {'job': 'Color Timer', 'name': 'Dale E. Grahn'},
 {'job': 'CG Painter', 'name': 'Robin Cooper'},
 {'job': 'Original Story', 'name': 'John Lasseter'},
 {'job': 'Original Story', 'name': 'Pete Docter'},
 {'job

In [32]:
df[df.original_title.str.contains('Les Mis')].head(1)

,adult,genres,original_language,original_title,popularity,production_companies,production_countries,release_date,revenue,spoken_languages,title,vote_average,vote_count,cast,crew
1772,False,"Crime, Drama, History, Romance",en,Les Misérables,12.238488,Columbia Pictures,United States of America,1998-05-01,14096321.0,"English, Français, Español",Les Misérables,7.1,157.0,"[{'character': 'Jean Valjean', 'gender': 'm', ...","[{'job': 'Director', 'name': 'Bille August'}, ..."


In [38]:
filtered_df = df[(df['release_date'] >= '2012-01-01') & (df['release_date'] <= '2013-12-31')]

In [47]:
filtered_df[filtered_df.original_title.str.contains('Les Mis')].cast.values[0][0]['name']

'Hugh Jackman'

In [79]:
filtered_df[filtered_df.original_title.str.contains('Silver Linings')].cast.values[0][1]['name']

'Jennifer Lawrence'